In [14]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen-2.5-VL")

In [15]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked."
    },
    
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
print(text_prompt)

<|im_start|>system
You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked.<|im_end|>



In [17]:
conversation = [

    {
        "role": "user",
        "content": [
            {"type": "video", "video": "/scratch2/tannicholas/data/videos/1.mp4"},
            {"type": "text", "text": "What happened in the video?"},
        ],
    },
    
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
print(text_prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|video_pad|><|vision_end|>What happened in the video?<|im_end|>



In [16]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked."
    },
    {
        "role": "user",
        "content": [
            {"type": "video", "video": "/scratch2/tannicholas/data/videos/1.mp4"},
            {"type": "text", "text": "What happened in the video?"},
        ],
    },
    {
        "role": "assistant",
        "content": "I'll describe what I observed in the video. Please share the video and I'll provide a detailed description."
    }
]
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
print(text_prompt)

<|im_start|>system
You are a helpful AI assistant that can understand and describe videos. Please provide clear and detailed descriptions of video content when asked.<|im_end|>
<|im_start|>user
<|vision_start|><|video_pad|><|vision_end|>What happened in the video?<|im_end|>
<|im_start|>assistant
I'll describe what I observed in the video. Please share the video and I'll provide a detailed description.<|im_end|>



In [5]:

import torch



input_tensor = torch.randn(2, 3)  # A random tensor of size 2x3

filled_tensor = torch.full_like(input_tensor, 0)  # Creates a tensor with the same size as input_tensor, filled with 0s



print(filled_tensor) 


tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [6]:
inputs = processor(
    text = text_prompt,
    padding=True,
    return_tensors="pt",
)

In [7]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [8]:
inputs['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])

In [11]:
from qwen_vl_utils.vision_process import process_vision_info
conversation = [
    {
        "role": "user", 
        "content": [
            {
                "type": "video", 
                "video": "../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/test/08December_2009_Tuesday_heute-4198.mp4",


                "fps": 25
            }
            ]
        }
    ]

_, video_input, video_kwargs = process_vision_info(conversation, return_video_kwargs=True)

In [12]:
video_input[0].shape

torch.Size([48, 3, 364, 308])

In [21]:
inputs = processor(
    videos = video_input,
    text = text_prompt,
    padding=True,
    return_tensors="pt",
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values_videos', 'video_grid_thw', 'second_per_grid_ts'])

In [22]:
print(inputs['pixel_values_videos'].shape)

torch.Size([13728, 1176])


In [20]:
print(inputs['video_grid_thw'])

tensor([[24, 26, 22]])
